In [ ]:
from pathlib import Path
import pandas as pd
import glob
import numpy as np
import pyarrow.parquet
import pyarrow as pa
import seaborn as sns
#import tqdm
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import gc
import os

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

### pathを何度も書くのが面倒なのでroot_pathを設定

In [ ]:
root_path = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/'

In [ ]:
train = pd.read_csv(root_path + 'train_labels.csv')
test = pd.read_csv(root_path + 'sample_submission.csv')
submission = pd.read_csv(root_path + 'sample_submission.csv')

## 特定の画像にアクセスしたい

In [ ]:
d = pydicom.read_file(root_path + 'train/00000/FLAIR/Image-1.dcm')
data = d.pixel_array # change pydicom data to numpy.ndarray
plt.imshow(data)

## 複数のDICOM画像を読みこんで1枚の連続した断層画像に変換する

In [ ]:
sample_dir = root_path + 'train/00000/FLAIR/'
num = len(glob.glob(sample_dir + '*.dcm'))
print(num)
dcms = []
for d, s, fl in os.walk(sample_dir):
    for fn in fl:
        if ".dcm" in fn.lower():
            dcms.append(os.path.join(d, fn))
ref_dicom = pydicom.read_file(dcms[0])
d_array = np.zeros((ref_dicom.Rows, ref_dicom.Columns, len(dcms)), dtype=ref_dicom.pixel_array.dtype)
for dcm in dcms:
    d = pydicom.read_file(dcm)
    d_array[:, :, dcms.index(dcm)] = d.pixel_array
print(d_array.shape)
plt.subplot(1, 3, 1)
plt.imshow(d_array[256, :, :])
plt.subplot(1, 3, 2)
plt.imshow(d_array[:, 256, :])
plt.subplot(1, 3, 3)
plt.imshow(d_array[:, :, 300])

In [ ]:
import pydicom
import numpy as np
import matplotlib.pyplot as plt
import sys
import glob

# DICOM ファイルを読み込み
files = []
print('DICOMファイルの場所: {}'.format(sample_dir))
for fname in glob.glob(sample_dir + '*.dcm'):
    files.append(pydicom.dcmread(fname))

# slicesというリストを作成。ファイルに、slicelocationという属性があれば追加していく。
slices = []
skipcount = 0
for f in files:
    if hasattr(f, 'SliceLocation'):
            if f.SeriesNumber>=0: #5回CTを撮影し、5回目のデータのみを抽出
                slices.append(f)
    else:
        skipcount = skipcount + 1

print("該当ファイル数: {}".format(len(slices)))
print("スライス位置がないファイル: {}".format(skipcount))

# スライスの順番をそろえる
slices = sorted(slices, key=lambda s: s.SliceLocation) # .SliceLocationで場所情報を取得
# len(slices) = 400

# アスペクト比を計算する
ps = slices[0].PixelSpacing    # ps = [0.571, 0.571] 1ピクセルの [y, x] 長さ
ss = slices[0].SliceThickness  # ss = "3.0" スライスの厚み
ax_aspect = ps[0]/ps[1]        # yの長さ/xの長さ =  1
sag_aspect = ss/ps[0]          # スライスの厚み/y軸方向への１ピクセルの長さ = 3/0.571 = 5.25
cor_aspect = ss/ps[1]          # スライスの厚み/x軸方向への１ピクセルの長さ = 3/0.571 = 5.25

# 空の3Dのnumpy arrayを作成する 
img_shape = list(slices[0].pixel_array.shape) # img_shape = [512, 512]
img_shape.insert(0,len(slices)) # img_shape = [270, 512, 512]  
img3d = np.zeros(img_shape) # 空のarrayを作る　np.zeros([270, 512, 512])

# 3Dのnumpy arrayを作る
for i, s in enumerate(slices):
    img2d = s.pixel_array    # img2d.shape = (512, 512) 
    img3d[i,:, :] = img2d    # img3d.shape = (270, 512, 512)

# プロットする
middle0 = img_shape[0]//2  #3d array の(z軸)頭尾軸中央　270/2 = 135
middle1 = img_shape[1]//2  #3d array の(y軸)上下軸中央　512/2 = 256
middle2 = img_shape[2]//2  #3d array の(x軸)左右軸中央　512/2 = 256

a1 = plt.subplot(1, 3, 1)
plt.imshow(img3d[middle0,:, :])
a1.set_aspect(ax_aspect)  # ax_aspect = 1

a2 = plt.subplot(1, 3, 2)
plt.imshow(img3d[:, middle1, :])
a2.set_aspect(cor_aspect) # cor_aspect = 5.25

a3 = plt.subplot(1, 3, 3)
plt.imshow(img3d[:, :, middle2]) # 
a3.set_aspect(sag_aspect) # sag_aspect = 5.25

plt.show()

統一されていない画像数をすべて３つずつにすれば分析が簡単になりそう
<br>画像があるのは4フォルダなので最大この方法だと12枚の写真、カラムを追加して分析すれば良さそう

In [51]:
%matplotlib inline
import matplotlib.pyplot as plt
from ipywidgets import interact
import numpy as np

param = len(slices)
    
def f(k):
    a1 = plt.subplot(1, 1, 1)
    plt.imshow(img3d[k, :, :])
    a1.set_aspect(ax_aspect)
    plt.show()

interact(f, k=(0,param-1) )

interactive(children=(IntSlider(value=199, description='k', max=399), Output()), _dom_classes=('widget-interac…

<function __main__.f(k)>